#딥러닝 사진 구분


In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 기본 데이터베이스에 해당하는 파일 디렉토리()
!mkdir datafile

mkdir: cannot create directory ‘datafile’: File exists


In [3]:
!mkdir image

mkdir: cannot create directory ‘image’: File exists


In [4]:
# static 디렉토리 생성
! mkdir static

mkdir: cannot create directory ‘static’: File exists


- 생성된 datafile 디렉토리에 n_.h5 딥러닝 학습 모델, classes.txt(클래스 분류 파일) 넣기

In [5]:
# 넘파이, 판다스 임포트
import numpy as np
import pandas as pd

In [6]:
# 텐서플로우 모델로더 임포트
from tensorflow.keras.models import load_model

In [7]:
# 데이터베이스 디렉토리 패스
file_path = './datafile/'

In [8]:
# 학습된 딥러닝 모델 로드

model = load_model(file_path + 'n_.h5')

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_10 (Dense)            (None, 256)               6422784   
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 149)               38293     
                                                                 
Total params: 21,176,789
Trainable params: 13,541,013
Non-trainable params: 7,635,776
__________________________________

In [9]:
# 추가 모듈 임포트
import json
import numpy as np
from PIL import Image

In [10]:
img_path = './image/'

In [11]:
# 사진 구분 함수 (사진 데이터에서 음식 이름 추출) 정의

def prd_img(filename):
    img_dir= img_path+'{}'.format(filename)
    with open(file_path+"classes.txt", "r") as fp:
        classes = json.load(fp)
    tst = Image.open(img_dir)
    tst = tst.convert('RGB')
    tst=tst.resize((224,224))
    tst=np.expand_dims(tst,axis=0)
    prd=model.predict(tst)
    prd=np.argmax(prd[0])
    name = classes[prd]
    return name

#음식정보 데이터 리드

- datafile 디렉토리에 K-Food_Recipe.csv, K-Food_info.csv, recipe_resource.csv 파일 넣기

In [12]:
data = pd.read_csv(file_path+'K-Food_info.csv', encoding = 'cp949', index_col='음식명').drop('구분', axis = 1).fillna('No data')

In [13]:
data2 = pd.read_csv(file_path+'./K-Food_Recipe.csv', encoding='cp949', index_col='RECIPE_NM_KO')

In [14]:
data3 = pd.read_csv(file_path+'./recipe_resource.csv', encoding='cp949').drop(['IRDNT_SN', 'IRDNT_CPCTY', 'IRDNT_TY_CODE', 'IRDNT_TY_NM'], axis = 1)

# 음식정보

In [15]:
# 음식명에 따른 요리명, 영문 요리명, 음식 설명 출력 함수 정의
def food_en(name):
    failtext = '요리를 찾지 못했습니다.'
    try:
        food_name = name
        food_name_en = data.loc[name]['영문']
        food_info_en = data.loc[name]['설명한글']

        return food_name, food_name_en, food_info_en
    except:
        food_name = name
        food_name_en = failtext
        food_info_en = failtext
        return food_name, food_name_en, food_info_en

# 알레르기정보

In [16]:
# 재료별 알레르기 정보
Allergie = {'메밀': ['메밀', '메밀가루'],
            '밀': ['밀가루', '밀', '강력분', '박력분', '중력분', '빵', '부침가루', '바게트', '칼국수', '국수', '소면', '면', '쫄면', '라면'],
            '대두': ['두부', '어묵', '콩나물'],
            '호두': ['호두'],
            '땅콩': ['땅콩', '땅콩버터'],
            '복숭아': ['복숭아'],
            '게': ['꽃게', '참게', '대게', '킹크랩'],
            '고등어': ['고등어'],
            '토마토': ['토마토'],
            '돼지고기': ['돼지고기', '갈비', '다진돼지고기', '베이컨', '돼지고기육수', '삼겹살', '햄'],
            '계란': ['계란', '달걀'],
            '우유': ['우유', '치즈', '치즈가루', '모짜렐라치즈'],
            '닭고기': ['닭고기'],
            '쇠고기': ['쇠고기', '소고기', '갈비', '쇠고기육수', '양지머리', '안심', '등심'],
            '새우': ['새우', '대하', '건새우', '새우젓', '생새우'],
            '홍합': ['홍합'],
            '전복': ['전복'],
            '굴': ['굴', '굴소스'],
            '조개류': ['바지락', '꼬막', '골뱅이', '소라'],
            '오징어': ['오징어', '진미채', '오징어젓갈'],
            '아황산': ['소시지', '소세지', '햄', '스팸', '육포','베이컨']}

In [17]:
# 음식명에 따른 음식 레시피, 레시피를 기반으로 알레르기 유발재료를 도출하는 함수 정의
def food_resource(name):
    failment = str('레시피를 찾을 수 없습니다.')
    try:
        rc_no = int(data2.loc[name]['RECIPE_ID'])
        resource = data3[data3.RECIPE_ID == rc_no]['IRDNT_NM'].tolist()

        result = resource
    except:
        result = failment
        
    try:
        alwar = set()
        for key, value in Allergie.items():
            alsource = set(value) & set(result)
            if alsource != set():
                alwar = alwar | alsource
            else:
                pass
    except:
        pass
    
    try:
        keys = []
        for key, value in Allergie.items():
            if alwar & set(value) != set():
                keys.append(key)
    except:
        pass
  
    if keys != []:
      return result, keys
    else:
      return result, keys

# 네이버 API 적용

In [18]:
  import requests
  import json
  import os
  import sys
  import urllib.request
  import shutil
  import time

In [19]:
# 음식명을 한국어 발음으로 mp3화하여 저장하는 함수 정의
def foodname_voice(name):
  client_id = "fr2pmzwe08"
  client_secret = "uzTbsKj83c57qTJNlHRbeIsTiwDdVoZLKDod8UBh"
  encText = urllib.parse.quote(name)
  data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
  url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode('utf-8'))
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      with open('foodname.mp3', 'wb') as f:
          f.write(response_body)
      source = 'foodname.mp3'
      destination = '/content/static/foodname.mp3'
      shutil.move(source, destination)
  else:
    errormsg = ("Error Code:" + rescode)
    return errormsg

In [20]:
# 입력된 텍스트와 희망 언어를 입력하여 텍스트를 희망 언어로 번역해주는 함수 정의
def translation(text, lng):
  client_id = "fr2pmzwe08"
  client_secret = "uzTbsKj83c57qTJNlHRbeIsTiwDdVoZLKDod8UBh"

  # lang = ko, ja, en, zh-CN, zh-TW, vi, th, es, fr, id, ru, de, it

  lang = lng
  encText = urllib.parse.quote(text)
  data = "source=ko&target={}&text=".format(lang) + encText
  url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"

  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode("utf-8"))
  rescode = response.getcode()


  if(rescode==200):
      response_body = response.read()
      response_encod = response_body.decode('utf-8')
      dict = json.loads(response_encod )
      mesge = dict['message']
      return(mesge['result']['translatedText'])

  else:
    errormsg = ("Error Code:" + rescode)
    return errormsg

# FastAPI


In [21]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

In [22]:
# 모듈 임포트
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

**API 화면 출력용 html 작성**

In [23]:
# 템플릿 디렉토리 생성
!mkdir templates

mkdir: cannot create directory ‘templates’: File exists


In [27]:
# UI 이미지 디렉토리 생성
!mkdir img

mkdir: cannot create directory ‘img’: File exists


In [24]:
title = '찾으신 음식정보입니다.'
fodna = '음식 이름'
fodnaen = '음식 이름(번역)'
foodin = '음식 정보'
allwarn = '알레르기 위험'
krdic = '한국어 발음'
interface = [title, fodna, fodnaen, foodin, allwarn, krdic]

- templates 디렉토리에 index.html 파일 넣기
- static 디렉토리에 style.css 파일 넣기
- img 디렉토리에 UI이미지 7종 넣기

아래 셀은 미사용 코드(기본 UI만 적용)(index.html 없을 때 사용)

In [25]:

%%writefile templates/input.html
<!DOCTYPE html>
<html>
<body>

<h2>요리</h2>

<form method="post" action="/topic" enctype="multipart/form-data">

  <input type="file" name="img" accept="image/*">

 

  <h2>언어</h2>

  <select name = 'lng'>
    <option type='text', value='en'>영어</option>
    <option type='text', value='ja'>일본어</option>
    <option type='text', value='vi'>베트남어</option>
    <option type='text', value='th'>태국어</option>
    <option type='text', value='es'>스페인어</option>
    <option type='text', value='fr'>프랑스어</option>
    <option type='text', value='id'>인도네시아어</option>
    <option type='text', value='ru'>러시아어</option>
    <option type='text', value='de'>독일어</option>
    <option type='text', value='it'>이탈리어어</option>
    <option type='text', value='zh-CN'>중국어(간체)</option>
    <option type='text', value='zh-TW'>중국어(번체)</option>
  </select>
  
  <input type="submit", Value=분석>

</form>

<p>분석 버튼을 누르시면 음식 정보 분석을 시작합니다.</p>

</body>
</html>

Overwriting templates/input.html


In [26]:
%%writefile templates/result_video.html
<html>
  <head>
    <h1 align = 'center'>{{inter1}}</h1>
  </head>
  <body>


      <audio>
        <source src= {{ fn }} type="audio/mpeg">
      </audio>
      <center>
        <img src= {{ image }} alt= 'imgnotfound', height="300" width="400"><br>
      </center>
    <p>
      <table width ="800" align = "center"><br>
        <tr>
            <td width ="200">{{inter2}}</td>
            <td>{{foodname}}</td>
        </tr>
        <tr>
            <td width ="200">{{inter3}}</td>
            <td>{{fooden}}</td>
        </tr>
        <tr>
          <td width ="200">{{inter4}}</td>
          <td>{{foodintr}}</td>
        </tr>
      {% if allergie != [] %}
        <tr>
          <td width ="200">※{{inter5}}</td>
          <td>{{allergie}}</td>
        </tr>    
      {% endif %}
        <tr>
          <td><br></td>
          <td><br></td>
        </tr>
        <tr>
         <td colspan="2">
          <h2>{{inter6}}</h2>

          <audio controls>
            <source src= {{ fn }} type="audio/mpeg">
            Your browser does not support the audio element.
          </audio><br>

          <a href="javascript:history.back()">뒤로가기</a>
          </td>
        </tr>
      </table></br>
    </p>



    <p>

    
    </p>

  </body>

</html>

Overwriting templates/result_video.html


**API 구성 및 실행**

In [35]:
# FastAPI
from fastapi.staticfiles import StaticFiles

app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')
app.mount('/image', StaticFiles(directory='image'), name='image')
app.mount('/img', StaticFiles(directory='img'), name='img')

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("input2.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_topic(request: Request, img: UploadFile = File(...), lng: str = Form(...)):
  file_location = f"image/{img.filename}"
  with open(file_location, "wb+") as file_object:
      file_object.write(img.file.read())
  # app.url_path_for('static', path='/{}'.format(img.filename))
  foodName = prd_img(img.filename)
  food_en(foodName)
  food_resource(foodName)
  foodname_voice(foodName)
  transres = translation(food_en(foodName)[2], lng)
  intertrance = []
  for translate in interface:
    interres = translation(translate, lng)
    intertrance.append(interres)
  allre = food_resource(foodName)[1]
  transal = []
  for transals in allre:
    allres = translation(transals, lng)
    transal.append(allres)

  food_en(foodName)[2],
  return templates.TemplateResponse("result_video.html", {"request": request,
                                                          "lng": lng,
                                                          "filename": img.filename,
                                                          'fn': '/static/foodname.mp3',
                                                          'foodname': prd_img(img.filename),
                                                          'fooden': food_en(foodName)[1],
                                                          'foodinen' : food_en(foodName)[2],
                                                          'foodsource' : food_resource(foodName)[0],
                                                          'allergie' : transal,
                                                          'foodintr' : transres,
                                                          'image': '/image/{}'.format(img.filename),
                                                          'inter1': intertrance[0],
                                                          'inter2': intertrance[1],
                                                          'inter3': intertrance[2],
                                                          'inter4': intertrance[3],
                                                          'inter5': intertrance[4],
                                                          'inter6': intertrance[5]
                                                          })

In [29]:
#ngrok authtoken 인증 (원활한 사용을 위해 개인 키를 이용할 것)
!ngrok authtoken 26zfEwqA01d1BVXYTKjLBxbTUDV_6nKybjK5amGTTzuo34T1u

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [30]:
app.url_path_for('static', path='/foodname.mp3')

'/static/foodname.mp3'

In [34]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://6d76-35-185-142-172.ngrok.io


INFO:     Started server process [5091]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     211.117.24.132:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /style.css HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /infoodd.PNG HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /INFOODD%20img.jpg HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET /icon-1294806.svg HTTP/1.1" 200 OK
INFO:     119.207.144.241:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.117.24.132:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.117.24.132:0 - "GET /style.css HTTP/1.1" 200 OK
INFO:     211.117.24.132:0 - "GET /icon-1294806.svg HTTP/1.1" 200 OK
INFO:     211.117.24.132:0 - "GET /INFOODD%20img.jpg HTTP/1.1" 200 OK
INFO:     211.117.24.132:0 - "GET /infoodd.PNG HTTP/1.1" 200 OK
INFO:     34.195.25.237:0 - "GET /ads.txt HTTP/1.1" 404 Not Found
INFO:     119.207.144.241:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2600:1f18:13dc:5c0c:eaef:5612:5451:77ce:0 - "GET / HTTP/1.1" 200 OK
INFO:     34.195.25.237:0 - "GET /IN

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5091]
